In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

In [2]:
###Load dataset
df = pd.read_csv('../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [3]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
customerID          7043 non-null object
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null object
Churn               7043 non-null object
dtypes: float64(1), int64(2), obj

In [5]:
#Drop customers with no charges/zero tenure Total charges to float
string_bool = df['TotalCharges'] > " "
df = df[string_bool]
df[['TotalCharges']] = df[['TotalCharges']].astype('float64')
print(len(df))

7032


In [6]:
df['senior_citizen'] = ['Yes' if x == 1 else 'No' for x in df['SeniorCitizen']]

In [7]:
#change SC to object, drop SC and ID
df.drop('SeniorCitizen', axis = 1, inplace = True)
df.drop('customerID', axis = 1, inplace = True)

In [8]:
##TotalCharges to float64, SeniorCitizen categorical?Binary?
df.describe()

,tenure,MonthlyCharges,TotalCharges
count,7032.000000,7032.000000,7032.000000
mean,32.421786,64.798208,2283.300441
std,24.545260,30.085974,2266.771362
min,1.000000,18.250000,18.800000
25%,9.000000,35.587500,401.450000
50%,29.000000,70.350000,1397.475000
75%,55.000000,89.862500,3794.737500
max,72.000000,118.750000,8684.800000


In [9]:
df['internet_service'] = ['No' if i == 'No' else 'Yes' for i in df.InternetService]

In [10]:
online_features = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                   'TechSupport', 'StreamingTV', 'StreamingMovies']
for i in df.columns:
    if df[i].dtype.kind not in 'biufc':
        print('{}: {}' .format(i, df[i].unique()))
        if 'No internet service' in df[i].unique():
            df[i] = [1 if x == 'Yes' else 0 for x in df[i]]
        elif len(df[i].unique()) == 2:
            t_df = pd.get_dummies(df[i], prefix = i,  drop_first = True)
            df[t_df.columns.values] = t_df
            df.drop(i, axis = 1, inplace = True)

gender: ['Female' 'Male']
Partner: ['Yes' 'No']
Dependents: ['No' 'Yes']
PhoneService: ['No' 'Yes']
MultipleLines: ['No phone service' 'No' 'Yes']
InternetService: ['DSL' 'Fiber optic' 'No']
OnlineSecurity: ['No' 'Yes' 'No internet service']
OnlineBackup: ['Yes' 'No' 'No internet service']
DeviceProtection: ['No' 'Yes' 'No internet service']
TechSupport: ['No' 'Yes' 'No internet service']
StreamingTV: ['No' 'Yes' 'No internet service']
StreamingMovies: ['No' 'Yes' 'No internet service']
Contract: ['Month-to-month' 'One year' 'Two year']
PaperlessBilling: ['Yes' 'No']
PaymentMethod: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
Churn: ['No' 'Yes']
senior_citizen: ['No' 'Yes']
internet_service: ['Yes' 'No']


In [11]:
df['multiple_lines'] = [1 if x == 'Yes' else 0 for x in df["MultipleLines"]]
df.drop('MultipleLines', axis = 1, inplace = True)

In [12]:
bins = pd.IntervalIndex.from_tuples([(0, 31), (31,66), (66, np.max(df['MonthlyCharges']))])
df['MonthlyCharges'] = pd.cut(df['MonthlyCharges'], bins, labels=["low", "medium", "high"])
df['MonthlyCharges'] = df['MonthlyCharges'].cat.codes

In [13]:
dummies = pd.get_dummies(df['InternetService']).drop('No', axis = 1)
df.drop('InternetService', axis = 1, inplace = True)
df = df.join(dummies)

In [14]:
dummies = pd.get_dummies(df['Contract'])
df.drop('Contract', axis = 1, inplace = True)
df = df.join(dummies)

In [15]:
dummies = pd.get_dummies(df['PaymentMethod'])
df.drop('PaymentMethod', axis = 1, inplace = True)
df = df.join(dummies)

In [16]:
df.to_csv('../data/processed/clean.csv', index = False)